In [1]:
import os 
import pandas as pd
import sys
import glob
import numpy

The goal of this notebook is to collate desired sequences for each clade in a directory titled by that clade. Have functions for exons, transcripts, and gene sequences. Welcome to the roundup! &#x1F920;

In [16]:
#first, find the files associated with final clade membership by using the gene table 
base='/global/scratch/users/chandlersutherland/e14/NLRCladeFinder/Atha_NLRome/'
gene_table=base+"Atha_NLRome_GeneTable.txt"
gene=pd.read_csv(gene_table,sep = '\t')
gene

,Gene,Clade_0,Clade_1,Clade_2,Clade_3,Clade,Ecotype,Allele,File,HV
0,7273_T439-R1,Int7764_78,Int7764_78_107_R_21,Int7764_78_107_R_21,NaN,Int7764_78_107_R_21,7273,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int7764...,0
1,7373_T332-R1,Int7764_78,Int7764_78_107_R_21,Int7764_78_107_R_21,NaN,Int7764_78_107_R_21,7373,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int7764...,0
2,9100_T487-R1,Int7764_78,Int7764_78_107_R_21,Int7764_78_107_R_21,NaN,Int7764_78_107_R_21,9100,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int7764...,0
3,1925_T547-R1,Int7764_78,Int7764_78_107_R_21,Int7764_78_107_R_21,NaN,Int7764_78_107_R_21,1925,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int7764...,0
4,7416_T304-R1,Int7764_78,Int7764_78_107_R_21,Int7764_78_107_R_21,NaN,Int7764_78_107_R_21,7416,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int7764...,0
...,...,...,...,...,...,...,...,...,...,...
7674,9100_T255-R1,Int14034_18,Int14034_18,NaN,NaN,Int14034_18,9100,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int14034_18...,0
7675,6906_T240-R1,Int14034_18,Int14034_18,NaN,NaN,Int14034_18,6906,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int14034_18...,0
7676,9669_T203-R1,Int14034_18,Int14034_18,NaN,NaN,Int14034_18,9669,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int14034_18...,0
7677,9721_T213-R1,Int14034_18,Int14034_18,NaN,NaN,Int14034_18,9721,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int14034_18...,0


In [3]:
#generate a dictionary with Clade names as the keys and representative genes as the values by getting each gene in the table associated with the clade
dictionary=dict()
clades=numpy.unique(gene['Clade'])
for clade in clades:
    print(clade)
    genes=list(gene[gene['Clade']==clade]['Gene'])
    dictionary[clade]=genes

Int10172_376_423_L_90_128_L_41
Int10172_376_423_L_90_128_R_49
Int10172_376_423_R_43
Int10172_376_424_R_11
Int10172_376_435_R_60
Int10172_376_495_R_128
Int10172_376_624_R_44
Int10547_34_50_L_17
Int10547_34_50_R_17
Int10580_55
Int10637_304_324_L_31
Int10637_304_324_R_203
Int10637_304_326_R_37
Int10637_304_362_R_19
Int10637_304_381_R_3
Int10637_304_546_R_11
Int10942_60
Int11003_58
Int11061_33_35_L_29
Int11061_33_35_R_4
Int11093_76_144_R_3
Int11093_76_90_L_19
Int11093_76_91_R_54
Int11170_39_55_L_37
Int11170_39_55_R_2
Int11208_40
Int11247_267_325_L_64
Int11247_267_326_R_108_147_L_56
Int11247_267_326_R_108_147_R_52_63_L_17
Int11247_267_326_R_108_147_R_52_64_R_9
Int11247_267_326_R_108_147_R_52_73_R_17
Int11247_267_326_R_108_147_R_52_90_R_2
Int11247_267_326_R_108_147_R_52_91_R_7
Int11247_267_434_R_35
Int11247_267_468_R_60
Int11514_34
Int11547_65_120_L_59
Int11547_65_120_R_6
Int11629_24_35_L_12
Int11629_24_36_R_11
Int11653_56
Int11708_423_513_L_319
Int11708_423_515_R_4
Int11708_423_518_R_3
Int1

In [4]:
#Define CDS roundup, which takes the items of this dictionary and writes a fasta file with all exon sequences to the clade folder
def CDS_roundup(clade, gene_list):
    #initialize some housekeeping, count being the number of found exons and found a set of gene names 
    count=0
    found=[]
    
    #get the clade name and initialize the new output directory 
    output_dir="/global/scratch/users/chandlersutherland/e14/popgen/clades/"+clade
    if not os.path.exists(output_dir): #this prevents a directory exists error
       os.makedirs(output_dir)
    
    #read in the gene list per clade, and clean up 
    clean_list=[]
    for i in gene_list:
        clean=i.replace('_1', '.1').replace('_2', '.2').replace('_3', '.3')
        clean_list.append(clean)

    # create output CDS file 
    g = open(output_dir+"/"+clade+'.CDS.fa', "w")

    # open the NLRome CDS file and search for gene names 
    #if found, write to new fasta file 
    with open("/global/scratch/users/chandlersutherland/e14/hvNLR/ArabidopsisRENSEQ/CDS/NLRome.CDS.fasta",'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
            for gene in clean_list:
                if gene in line:
                    count=count+1
                    found.append(gene)
                    g.writelines(line)
                    g.writelines(lines[lines.index(line)+1])
    
    #special treatment for Col-0 because of the full gene names 
    with open("/global/scratch/users/chandlersutherland/e14/popgen/Col0_Exons.fa",'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
                for gene in clean_list:
                    if gene.startswith('ATHALIANA'):
                        short=gene.replace('ATHALIANA_', '')
                        if short in line: 
                            count=count+1
                            found.append(gene)
                            g.writelines(line)
                            g.writelines(lines[lines.index(line)+1])
    
    #sanity check. The found genes should be the same as the input genes. 
    print('found '+str(count)+' transcripts')
    s=set(found)
    q=set(list(clean_list))
    if s == q:
          print('found all genes supplied')
    else:
          print('error, some genes missing')
          temp3 = [x for x in q if x not in s]
          print(str(temp3)+' not found')
    
    g.close()
    print('CDS file written to '+output_dir+"/"+clade+'.CDS.fa')

In [139]:
for k, v in dictionary.items():
    CDS_roundup(k, v)

found 179 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_423_L_90_128_L_41/Int10172_376_423_L_90_128_L_41.CDS.fa
found 233 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_423_L_90_128_R_49/Int10172_376_423_L_90_128_R_49.CDS.fa
found 180 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_423_R_43/Int10172_376_423_R_43.CDS.fa
found 46 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_424_R_11/Int10172_376_424_R_11.CDS.fa
found 250 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_435_R_60/Int10172_376_435_R_60.CDS.fa
found 503 transcripts
found all genes supplied
CDS file written to /global/scrat

found 287 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12133_61/Int12133_61.CDS.fa
found 126 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12194_239_292_L_69_103_R_28/Int12194_239_292_L_69_103_R_28.CDS.fa
found 61 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12194_239_292_L_69_77_L_15/Int12194_239_292_L_69_77_L_15.CDS.fa
found 133 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12194_239_292_L_69_78_R_26/Int12194_239_292_L_69_78_R_26.CDS.fa
found 258 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12194_239_292_R_56/Int12194_239_292_R_56.CDS.fa
found 285 transcripts
found all genes supplied
CDS file written to /global/scratch/use

found 53 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13739_48_54_L_9/Int13739_48_54_L_9.CDS.fa
found 192 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13739_48_54_R_39/Int13739_48_54_R_39.CDS.fa
found 301 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13787_62/Int13787_62.CDS.fa
found 313 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13848_61/Int13848_61.CDS.fa
found 212 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13909_42/Int13909_42.CDS.fa
found 286 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13950_57/Int13950_57.CDS.fa
found 95 transcripts
found al

found 6 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7765_208_264_R_3/Int7765_208_264_R_3.CDS.fa
found 13 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7765_208_266_R_9/Int7765_208_266_R_9.CDS.fa
found 179 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7765_208_274_R_142/Int7765_208_274_R_142.CDS.fa
found 88 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7973_410_444_L_50/Int7973_410_444_L_50.CDS.fa
found 64 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7973_410_445_R_57/Int7973_410_445_R_57.CDS.fa
found 502 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int797

found 226 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9457_231_369_R_54/Int9457_231_369_R_54.CDS.fa
found 165 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9687_297_319_L_40/Int9687_297_319_L_40.CDS.fa
found 236 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9687_297_321_R_58/Int9687_297_321_R_58.CDS.fa
found 183 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9687_297_380_R_42/Int9687_297_380_R_42.CDS.fa
found 122 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9687_297_382_R_27/Int9687_297_382_R_27.CDS.fa
found 84 transcripts
found all genes supplied
CDS file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/

In [28]:
#Define Gene roundup, which takes the items of this dictionary and writes a fasta file with all Gene sequences to the clade folder
def Gene_roundup(clade, gene_list):
    #initialize some housekeeping, count being the number of found exons and found a set of gene names 
    count=0
    found=[]
    
    #get the clade name and initialize the new output directory 
    output_dir="/global/scratch/users/chandlersutherland/e14/popgen/clades/"+clade
    if not os.path.exists(output_dir): #this prevents a directory exists error
       os.makedirs(output_dir)
    
    #read in the gene list per clade, and clean up 
    clean_list=[]
    for i in gene_list:
        clean=i.replace('_T', '_G').replace('-R1', '').replace('_1', '.1').replace('_2', '.2').replace('_3', '.3')
        clean_list.append(clean)

    # create output CDS file 
    g = open(output_dir+"/"+clade+'.GeneSequence_test.fa', "w")

    # open the NLRome CDS file and search for gene names 
    #if found, write to new fasta file 
    with open("/global/scratch/users/chandlersutherland/e14/hvNLR/ArabidopsisRENSEQ/GeneSequences/NLRome_Gene_Sequences.respectify.fa",'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
            for gene in clean_list:
                if gene in line:
                    count=count+1
                    found.append(gene)
                    g.writelines(line)
                    g.writelines(lines[lines.index(line)+1])
    
    #special treatment for Col-0 because of the full gene names 
    with open("/global/scratch/users/chandlersutherland/e14/popgen/Col0_GeneSequences.fa",'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
                for gene in clean_list:
                    if gene.startswith('ATHALIANA'):
                        short=gene.replace('ATHALIANA_', '').replace('_1', '').replace('.1', '')
                        if short in line: 
                            count=count+1
                            found.append(gene)
                            g.writelines(line)
                            g.writelines(lines[lines.index(line)+1])
    
    #sanity check. The found genes should be the same as the input genes. 
    print('found '+str(count)+' genes')
    s=set(found)
    q=set(list(clean_list))
    if s == q:
          print('found all genes supplied')
    else:
          print('error, some genes missing')
          temp3 = [x for x in q if x not in s]
          print(str(temp3)+' not found')
    
    g.close()
    print('Gene file written to '+output_dir+"/"+clade+'.GeneSequence.fa')

In [153]:
for k, v in dictionary.items():
    Gene_roundup(k, v)

found 41 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_423_L_90_128_L_41/Int10172_376_423_L_90_128_L_41.GeneSequence.fa
found 49 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_423_L_90_128_R_49/Int10172_376_423_L_90_128_R_49.GeneSequence.fa
found 43 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_423_R_43/Int10172_376_423_R_43.GeneSequence.fa
found 11 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_424_R_11/Int10172_376_424_R_11.GeneSequence.fa
found 60 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10172_376_435_R_60/Int10172_376_435_R_60.GeneSequence.fa
found 128 genes
found all genes supplied
Gene file written to /g

found 9 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int11708_423_813_R_9/Int11708_423_813_R_9.GeneSequence.fa
found 61 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12133_61/Int12133_61.GeneSequence.fa
found 28 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12194_239_292_L_69_103_R_28/Int12194_239_292_L_69_103_R_28.GeneSequence.fa
found 15 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12194_239_292_L_69_77_L_15/Int12194_239_292_L_69_77_L_15.GeneSequence.fa
found 26 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12194_239_292_L_69_78_R_26/Int12194_239_292_L_69_78_R_26.GeneSequence.fa
found 56 genes
found all genes supplied
Gene file written to /global/scra

found 23 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13684_55_74_L_23/Int13684_55_74_L_23.GeneSequence.fa
found 32 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13684_55_74_R_32/Int13684_55_74_R_32.GeneSequence.fa
found 9 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13739_48_54_L_9/Int13739_48_54_L_9.GeneSequence.fa
found 39 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13739_48_54_R_39/Int13739_48_54_R_39.GeneSequence.fa
found 62 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13787_62/Int13787_62.GeneSequence.fa
found 61 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int13848_61/Int1384

found 21 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7764_78_107_R_21/Int7764_78_107_R_21.GeneSequence.fa
found 43 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7765_208_251_L_43/Int7765_208_251_L_43.GeneSequence.fa
found 6 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7765_208_252_R_6/Int7765_208_252_R_6.GeneSequence.fa
found 5 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7765_208_260_R_5/Int7765_208_260_R_5.GeneSequence.fa
found 3 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int7765_208_264_R_3/Int7765_208_264_R_3.GeneSequence.fa
found 9 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/In

found 59 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9399_59/Int9399_59.GeneSequence.fa
found 57 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9457_231_249_L_57/Int9457_231_249_L_57.GeneSequence.fa
found 13 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9457_231_251_R_13/Int9457_231_251_R_13.GeneSequence.fa
found 50 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9457_231_264_R_50/Int9457_231_264_R_50.GeneSequence.fa
found 56 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9457_231_313_R_56/Int9457_231_313_R_56.GeneSequence.fa
found 1 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int9457_231

In [166]:
#sanity check 
len(numpy.unique(gene['Clade']))
#same number of directories as clades
genes=list(gene[gene['Clade']=='Int10637_304_324_R_203']['Gene'])
#sort(genes)
genes.sort()
genes

['10015_T018-R1',
 '10015_T023-R1',
 '10015_T024-R1',
 '108_T029-R1',
 '108_T033-R1',
 '108_T178-R1',
 '1925_T050-R1',
 '1925_T055-R1',
 '1925_T068-R1',
 '1925_T723-R1',
 '5784_T149_1-R1',
 '5784_T149_2-R1',
 '6899_T001-R1',
 '6899_T013_2-R1',
 '6906_T003-R1',
 '6906_T005-R1',
 '6906_T016_1-R1',
 '6906_T023-R1',
 '6906_T109-R1',
 '6906_T110-R1',
 '6909_T013_1-R1',
 '6909_T014_1-R1',
 '6909_T014_2-R1',
 '6909_T039-R1',
 '6909_T040_2-R1',
 '6924_T071-R1',
 '6924_T074-R1',
 '6924_T090-R1',
 '6924_T510-R1',
 '6939_T051-R1',
 '6939_T052-R1',
 '6939_T053-R1',
 '6939_T266-R1',
 '6939_T268-R1',
 '6939_T521-R1',
 '6974_T100-R1',
 '6974_T109-R1',
 '6974_T112-R1',
 '6981_T082_2-R1',
 '7058_T026-R1',
 '7111_T067-R1',
 '7111_T069-R1',
 '7111_T074-R1',
 '7111_T093-R1',
 '7111_T285-R1',
 '7167_T031-R1',
 '7167_T035-R1',
 '7167_T131-R1',
 '7186_T025-R1',
 '7186_T026_2-R1',
 '7186_T030-R1',
 '7186_T032-R1',
 '7186_T077-R1',
 '7186_T179-R1',
 '7186_T225-R1',
 '7186_T370-R1',
 '7213_T207-R1',
 '7273_T011

Input strategy change, get the Gene names from the alignment and use that to build the FASTA to ensure they are exactly the same 

In [4]:
#generate a dictionary with Clade names as the keys and representative genes as the values 
dictionary2=dict()
gene=gene.astype({'File':'str'})
files=numpy.unique(gene['File'])

cleanedfiles = [x for x in files if str(x) != 'nan']

for file in cleanedfiles:    
    genes=[]
    with open("/global/scratch/users/chandlersutherland/e14/NLRCladeFinder/Atha_NLRome/"+file,'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
            if line.startswith('>'):
                genes.append(line.replace('>', '').replace(' \n', ''))
                
    
    clade=file.split('/')[-1].replace('.afa', '')
    dictionary2[clade]=genes

dictionary2

{'Int10172_376_423_R_43': ['9869_T167-R1',
  '1925_T284-R1',
  '9837_T292_2-R1',
  '9792_T206-R1',
  '6906_T353-R1',
  '9887_T189-R1',
  '7067_T079-R1',
  '7167_T140-R1',
  '9762_T279-R1',
  '9545_T187-R1',
  '7416_T450-R1',
  '9947_T134-R1',
  '6909_T227-R1',
  '7111_T189-R1',
  '9944_T174_1-R1',
  '7413_T214-R1',
  '9784_T257-R1',
  '9557_T122-R1',
  '9871_T179_2-R1',
  '9542_T194_2-R1',
  '9518_T187_2-R1',
  '9721_T266-R1',
  '6899_T154-R1',
  '9583_T210_2-R1',
  '9550_T318_1-R1',
  'ATHALIANA_AT5G41550_1',
  '9100_T213-R1',
  '9597_T183-R1',
  '6939_T154-R1',
  '6981_T240-R1',
  '9543_T256_2-R1',
  '9600_T163_1-R1',
  '9879_T211-R1',
  '6924_T186-R1',
  '9549_T140-R1',
  '9554_T192_1-R1',
  '9332_T176_2-R1',
  '7373_T068-R1',
  '5784_T164-R1',
  '9536_T100_1-R1',
  '9610_T149-R1',
  '5993_T135_1-R1',
  '9654_T156-R1'],
 'Int10172_376_424_R_11': ['9554_T192_2-R1',
  '6981_T238-R1',
  '7328_T412-R1',
  '9944_T174_2-R1',
  '9542_T194_1-R1',
  '9871_T179_1-R1',
  '9543_T256_1-R1',
  '9

In [5]:
#get the Col0 genes so I can pull sequences from phytozome 
all_genes=list(dictionary2.values())
flat_list = [item for sublist in all_genes for item in sublist]
for gene in flat_list:
    if gene.startswith("ATHALIANA"):
        col0=gene.replace("ATHALIANA_", '').strip("_1")
        print(col0)

AT5G41550
AT5G38850
AT5G41740
AT5G41750
AT5G41540
AT4G16900
AT4G16960
AT4G16940
AT4G16920
AT4G16890
AT4G16860
AT4G16950
AT5G51630
AT5G46450
AT5G46260
AT5G46520
AT5G46510
AT5G46270
AT5G18360
AT3G44400
AT3G44480
AT3G44670
AT3G44630
AT3G25510
AT5G11250
AT1G69550
AT3G04220
AT5G38340
AT5G38350
AT1G65850
AT5G18370
AT5G18350
AT1G27180
AT1G27170
AT3G51560
AT5G17890
AT5G45050
AT5G45260
AT5G45210
AT4G19510
AT5G45060
AT5G45250
AT4G19530
AT5G17880
AT5G44870
AT5G45230
AT5G66910
AT5G66900
AT4G33300
AT5G04720
AT5G47280
AT3G46710
AT3G46730
AT3G46530
AT1G53350
AT5G35450
AT5G48620
AT5G43470
AT1G10920
AT1G59780
AT1G59620
AT1G58410
AT1G58400
AT1G59218
AT1G58848
AT1G59124
AT1G58807
AT1G58602
AT1G58390
AT3G14470
AT3G14460
AT1G12290
AT4G10780
AT1G63350
AT1G62630
AT1G63360
AT1G12210
AT1G12280
AT1G61180
AT1G61190
AT1G61310
AT1G61300
AT1G12220
AT1G51480
AT1G15890
AT5G05400
AT4G27190
AT4G27220
AT4G19050
AT5G40100
AT1G17600
AT1G72860
AT5G48770
AT1G72840
AT4G09430
AT1G63730
AT4G14370
AT5G58120
AT1G63740
AT1G64070


In [43]:
#Gene roundup sandbox to get the same name as in the alignment file 
def Gene_roundup_test(clade, gene_list):
    #initialize some housekeeping, count being the number of found exons and found a set of gene names 
    count=0
    found=[]
    
    #get the clade name and initialize the new output directory 
    output_dir="/global/scratch/users/chandlersutherland/e14/popgen/clades/"+clade
    if not os.path.exists(output_dir): #this prevents a directory exists error
       os.makedirs(output_dir)

    # create output Gene Sequences file 
    g = open(output_dir+"/"+clade+'.GeneSequence_test.fa', "w")

    # open the NLRome CDS file and search for gene names 
    #if found, write to new fasta file 
    with open("/global/scratch/users/chandlersutherland/e14/hvNLR/ArabidopsisRENSEQ/GeneSequences/NLRome_Gene_Sequences.respectify.fa",'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
            for gene in gene_list:
                clean=gene.replace('_T', '_G').replace('-R1', '').replace('_1', '.1').replace('_2', '.2').replace('_3', '.3')
                if clean in line:
                    count=count+1
                    found.append(gene)
                    g.writelines('>'+gene+'\n')
                    g.writelines(lines[lines.index(line)+1])
    
    #special treatment for Col-0 because of the full gene names 
    with open("/global/scratch/users/chandlersutherland/e14/popgen/Col0_GeneSequences.fa",'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
                for gene in gene_list:
                    if gene.startswith('ATHALIANA'):
                        short=gene.replace('ATHALIANA_', '').replace('_1', '').replace('.1', '')
                        if short in line: 
                            count=count+1
                            found.append(gene)
                            g.writelines('>'+gene+'\n')
                            g.writelines(lines[lines.index(line)+1])
    
    #sanity check. The found genes should be the same as the input genes. 
    print('found '+str(count)+' genes')
    s=set(found)
    q=set(list(gene_list))
    if s == q:
          print('found all genes supplied')
    else:
          print('error, some genes missing')
          temp3 = [x for x in q if x not in s]
          print(str(temp3)+' not found')
    
    g.close()
    print('Gene file written to '+output_dir+"/"+clade+'.GeneSequence.fa')

In [44]:
Gene_roundup_test(clade, dictionary2[clade])

found 34 genes
found all genes supplied
Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12822_34/Int12822_34.GeneSequence.fa


Switch strategy part 2: get the CDS together as single sequences to try not to break palnal2

In [6]:
#input: a clade name, and the list of genes found in the alignment in Daniil's format 
#output: a fasta file written, with Daniil gene IDs. Exons are concatenated to form the transcript sequence. 
#update: for col0, use tair10 transcripts 
def transcript_roundup(clade, gene_list):
    #initialize some housekeeping, count being the number of found exons and found a set of gene names 
    exons=[]
    found=[]

    #get the clade name and initialize the new output directory 
    output_dir="/global/scratch/users/chandlersutherland/e14/popgen/clades/"+clade
    if not os.path.exists(output_dir): #this prevents a directory exists error
        os.makedirs(output_dir)

    # create output CDS file 
    g = open(output_dir+"/"+clade+'.transcript.fa', "w")

    for gene in gene_list:
        #special treatment for Col-0 because of the full gene names 
        count=0
        if gene.startswith('ATHALIANA'):
            #using just the primary transcript
            clean=gene.replace('ATHALIANA_', '').replace('_1', '')+'.1'
            transcript=[]
            line_number=0

            with open("/global/scratch/users/chandlersutherland/e14/popgen/Col0_CDS_tair10.fa",'r') as text_file:
                lines = text_file.readlines()
                for line in lines:
                    line_number += 1
                    if line.find(clean) != -1: 
                        transcript.append(lines[line_number].strip('\n'))
                        count += 1
                        
            found.append(gene)
            exons.append([gene, count])
            g.writelines('>'+gene+'\n')
            g.writelines(''.join(transcript)+'\n')
         
        #everything else, search the NLRome CDS file 
        else:
            clean=gene.replace('_1', '.1').replace('_2', '.2').replace('_3', '.3')
            transcript=[]
            line_number=0

            with open("/global/scratch/users/chandlersutherland/e14/hvNLR/ArabidopsisRENSEQ/CDS/NLRome.CDS.fasta",'r') as text_file:
                lines = text_file.readlines()
                for line in lines:
                    line_number += 1
                    if line.find(clean) != -1: 
                        transcript.append(lines[line_number].strip('\n'))
                        count += 1

            found.append(gene)
            exons.append([gene, count])
            g.writelines('>'+gene+'\n')
            g.writelines(''.join(transcript)+'\n')

    g.close()
    
    #check that output transcripts matches number of input 
    s=set(found)
    q=set(list(gene_list))
    if s == q:
          print('found all genes supplied; '+str(len(s))+' genes\n\n')
    else:
          print('error, some genes missing')
          temp3 = [x for x in q if x not in s]
          print(str(temp3)+' not found')
    print('Exons added per gene: ')
    print(exons)
    print('\n\n')
    
    g.close()
    print('Gene file written to '+output_dir+"/"+clade+'.transcript.fa')

In [9]:
clade='Int10637_304_324_R_203'
gene_list=dictionary2[clade]
transcript_roundup(clade, gene_list)

found all genes supplied; 203 genes


Exons added per gene: 
[['9600_T065-R1', 8], ['9550_T090_2-R1', 7], ['9869_T024-R1', 3], ['9669_T118-R1', 6], ['7213_T207-R1', 2], ['9764_T051-R1', 15], ['1925_T723-R1', 5], ['6899_T001-R1', 11], ['10015_T023-R1', 6], ['9100_T098-R1', 7], ['7373_T001-R1', 9], ['9887_T032-R1', 3], ['9597_T134-R1', 6], ['9654_T575-R1', 6], ['7186_T077-R1', 6], ['9784_T009-R1', 5], ['9134_T017-R1', 7], ['9597_T455-R1', 7], ['9518_T027-R1', 7], ['9100_T590-R1', 2], ['6939_T268-R1', 6], ['7111_T067-R1', 7], ['6906_T005-R1', 6], ['7308_T308-R1', 6], ['7186_T225-R1', 9], ['9100_T036-R1', 8], ['ATHALIANA_AT4G16960_1', 1], ['7111_T069-R1', 7], ['9100_T079-R1', 9], ['7186_T370-R1', 7], ['9332_T004-R1', 14], ['9944_T254-R1', 8], ['9721_T027-R1', 14], ['9545_T127-R1', 5], ['7273_T021-R1', 8], ['7273_T017-R1', 7], ['7186_T026_2-R1', 5], ['9784_T058-R1', 7], ['6924_T074-R1', 8], ['9887_T030_1-R1', 7], ['6906_T023-R1', 5], ['9550_T047-R1', 9], ['9536_T199-R1', 6], ['6924_T510-R1'

In [7]:
for k, v in dictionary2.items():
    transcript_roundup(k, v)

found all genes supplied; 43 genes


Exons added per gene: 
[['9869_T167-R1', 4], ['1925_T284-R1', 4], ['9837_T292_2-R1', 4], ['9792_T206-R1', 4], ['6906_T353-R1', 4], ['9887_T189-R1', 4], ['7067_T079-R1', 4], ['7167_T140-R1', 8], ['9762_T279-R1', 4], ['9545_T187-R1', 4], ['7416_T450-R1', 4], ['9947_T134-R1', 4], ['6909_T227-R1', 4], ['7111_T189-R1', 6], ['9944_T174_1-R1', 4], ['7413_T214-R1', 5], ['9784_T257-R1', 4], ['9557_T122-R1', 4], ['9871_T179_2-R1', 4], ['9542_T194_2-R1', 4], ['9518_T187_2-R1', 4], ['9721_T266-R1', 4], ['6899_T154-R1', 4], ['9583_T210_2-R1', 4], ['9550_T318_1-R1', 4], ['ATHALIANA_AT5G41550_1', 1], ['9100_T213-R1', 4], ['9597_T183-R1', 4], ['6939_T154-R1', 4], ['6981_T240-R1', 4], ['9543_T256_2-R1', 4], ['9600_T163_1-R1', 4], ['9879_T211-R1', 4], ['6924_T186-R1', 4], ['9549_T140-R1', 4], ['9554_T192_1-R1', 4], ['9332_T176_2-R1', 4], ['7373_T068-R1', 4], ['5784_T164-R1', 4], ['9536_T100_1-R1', 4], ['9610_T149-R1', 4], ['5993_T135_1-R1', 5], ['9654_T156-R1', 4]]



found all genes supplied; 31 genes


Exons added per gene: 
[['9597_T338-R1', 4], ['5993_T089-R1', 6], ['9764_T044-R1', 9], ['6899_T013_1-R1', 9], ['7186_T202-R1', 6], ['9784_T617-R1', 4], ['9784_T067-R1', 2], ['7273_T014-R1', 5], ['7167_T049-R1', 8], ['ATHALIANA_AT4G16900_1', 1], ['9784_T001-R1', 10], ['7111_T492-R1', 5], ['9550_T290-R1', 10], ['9550_T090_1-R1', 7], ['6981_T221-R1', 6], ['9944_T039-R1', 6], ['6981_T082_1-R1', 7], ['9654_T232-R1', 8], ['6924_T087-R1', 5], ['7416_T016-R1', 5], ['7213_T051-R1', 9], ['7111_T090-R1', 10], ['108_T039-R1', 2], ['9545_T053-R1', 8], ['9554_T021-R1', 9], ['9100_T031-R1', 6], ['1925_T072-R1', 5], ['9784_T089-R1', 6], ['7067_T032-R1', 9], ['7373_T007-R1', 8], ['6909_T021-R1', 6]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int10637_304_324_L_31/Int10637_304_324_L_31.transcript.fa
found all genes supplied; 203 genes


Exons added per gene: 
[['9600_T065-R1', 8], ['9550_T090_2-R1', 7], ['9869_T024-R1', 3], ['9

found all genes supplied; 4 genes


Exons added per gene: 
[['9784_T505-R1', 5], ['7186_T379-R1', 5], ['9762_T530-R1', 5], ['9610_T528-R1', 5]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int11061_33_35_R_4/Int11061_33_35_R_4.transcript.fa
found all genes supplied; 19 genes


Exons added per gene: 
[['7186_T361-R1', 3], ['9792_T416-R1', 4], ['9869_T381-R1', 6], ['9597_T486-R1', 4], ['9764_T360-R1', 5], ['7373_T264-R1', 5], ['7416_T208-R1', 5], ['108_T481-R1', 5], ['9669_T338-R1', 4], ['1925_T588-R1', 5], ['9784_T490-R1', 4], ['7213_T328-R1', 4], ['7273_T288-R1', 4], ['9533_T336-R1', 5], ['7413_T255-R1', 4], ['5993_T357-R1', 4], ['6981_T380-R1', 5], ['7067_T346-R1', 5], ['6899_T392-R1', 4]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int11093_76_90_L_19/Int11093_76_90_L_19.transcript.fa
found all genes supplied; 54 genes


Exons added per gene: 
[['9536_T008-R1', 5], ['9871_T166-R1', 5], ['9543_T049-R1', 5], [

found all genes supplied; 6 genes


Exons added per gene: 
[['9533_T432-R1', 4], ['9536_T236-R1', 4], ['9947_T378-R1', 4], ['9542_T425-R1', 4], ['9887_T314-R1', 4], ['7417_T332-R1', 4]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int11547_65_120_R_6/Int11547_65_120_R_6.transcript.fa
found all genes supplied; 12 genes


Exons added per gene: 
[['6899_T330-R1', 5], ['9536_T229-R1', 7], ['6924_T386-R1', 6], ['7067_T319-R1', 6], ['9549_T339-R1', 7], ['108_T245-R1', 7], ['9554_T094-R1', 8], ['9784_T164-R1', 6], ['7288_T325-R1', 6], ['9792_T182-R1', 8], ['9557_T177-R1', 6], ['9654_T548-R1', 6]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int11629_24_35_L_12/Int11629_24_35_L_12.transcript.fa
found all genes supplied; 11 genes


Exons added per gene: 
[['9792_T184-R1', 3], ['7328_T171-R1', 2], ['9554_T092-R1', 2], ['9536_T416-R1', 2], ['7288_T505-R1', 4], ['6899_T454-R1', 4], ['9557_T179-R1', 2], ['108_T243_2-R1', 2]

found all genes supplied; 59 genes


Exons added per gene: 
[['7416_T091-R1', 5], ['7186_T231-R1', 5], ['6906_T249_1-R1', 5], ['108_T355_1-R1', 5], ['9545_T167-R1', 4], ['9944_T153-R1', 5], ['9518_T193-R1', 5], ['9947_T114-R1', 3], ['9542_T228-R1', 5], ['7413_T247_2-R1', 5], ['7213_T145-R1', 5], ['9871_T246-R1', 5], ['7415_T203-R1', 5], ['6981_T267-R1', 4], ['9554_T180-R1', 5], ['9533_T232-R1', 5], ['7417_T114-R1', 5], ['7067_T167-R1', 4], ['108_T335-R1', 11], ['9550_T295_2-R1', 6], ['6924_T230-R1', 4], ['6974_T236-R1', 5], ['5993_T221-R1', 5], ['9669_T270-R1', 5], ['9762_T262-R1', 5], ['7058_T143-R1', 4], ['9100_T245-R1', 4], ['7308_T118-R1', 3], ['6909_T205-R1', 4], ['9580_T212-R1', 4], ['9600_T230-R1', 4], ['9597_T290-R1', 5], ['1925_T364-R1', 5], ['9879_T198-R1', 5], ['9583_T228-R1', 4], ['7111_T416-R1', 4], ['9784_T251-R1', 4], ['9837_T181-R1', 5], ['9549_T129-R1', 5], ['9792_T289-R1', 4], ['9543_T237-R1', 5], ['7167_T188-R1', 4], ['7322_T472-R1', 5], ['9557_T214-R1', 4], ['9764_T

...

found all genes supplied; 38 genes


Exons added per gene: 
[['9542_T190-R1', 1], ['9554_T221-R1', 1], ['9597_T244-R1', 1], ['7416_T226-R1', 1], ['9944_T139-R1', 1], ['9557_T184-R1', 1], ['9764_T252-R1', 1], ['7058_T175-R1', 1], ['9537_T202-R1', 1], ['6909_T155-R1', 1], ['7167_T423-R1', 1], ['9134_T164-R1', 1], ['9837_T186-R1', 1], ['1925_T350-R1', 1], ['9580_T231-R1', 1], ['9536_T119-R1', 1], ['7273_T260-R1', 1], ['7322_T172-R1', 2], ['9610_T214-R1', 1], ['9669_T207-R1', 1], ['9533_T261-R1', 1], ['6924_T469-R1', 1], ['9545_T192-R1', 1], ['6899_T191-R1', 1], ['9332_T186-R1', 1], ['7186_T132-R1', 1], ['9871_T217-R1', 1], ['7288_T236-R1', 1], ['9879_T221-R1', 1], ['9947_T086-R1', 1], ['7417_T106-R1', 1], ['6906_T223-R1', 1], ['9887_T201-R1', 1], ['ATHALIANA_AT5G43730_1', 1], ['9100_T234-R1', 1], ['9550_T282-R1', 1], ['7308_T115-R1', 1], ['9762_T234-R1', 1]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int8443_258_351_L_88_120_L_38/Int8443_258_351_L_8

found all genes supplied; 40 genes


Exons added per gene: 
[['9537_T110-R1', 5], ['9762_T111-R1', 6], ['5784_T110-R1', 5], ['7067_T150-R1', 6], ['9947_T037-R1', 5], ['6939_T145-R1', 5], ['7186_T089-R1', 5], ['7111_T178-R1', 5], ['9536_T041-R1', 5], ['7213_T087-R1', 8], ['9610_T095-R1', 5], ['9549_T080-R1', 6], ['6974_T141-R1', 5], ['7373_T038-R1', 7], ['7322_T409-R1', 6], ['5993_T080-R1', 5], ['9545_T102-R1', 6], ['7328_T266-R1', 5], ['10015_T087-R1', 5], ['7308_T028-R1', 6], ['6899_T275-R1', 5], ['9887_T063-R1', 6], ['9557_T190-R1', 5], ['9721_T134-R1', 5], ['6906_T158-R1', 5], ['7415_T223-R1', 6], ['9542_T091-R1', 5], ['9764_T167-R1', 5], ['9792_T180-R1', 6], ['9332_T144-R1', 5], ['1925_T182-R1', 5], ['9871_T101-R1', 5], ['7396_T124-R1', 5], ['108_T198-R1', 5], ['9669_T137-R1', 6], ['9580_T135-R1', 6], ['7413_T107-R1', 5], ['6924_T149-R1', 6], ['9597_T103-R1', 5], ['7167_T101-R1', 5]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int11208_40/Int1

found all genes supplied; 16 genes


Exons added per gene: 
[['6974_T527-R1', 4], ['9871_T539-R1', 5], ['7328_T578-R1', 5], ['9580_T564-R1', 5], ['9792_T550-R1', 5], ['7413_T411-R1', 4], ['9887_T478-R1', 5], ['9554_T449-R1', 5], ['9784_T503-R1', 5], ['9947_T463-R1', 5], ['9944_T429-R1', 5], ['7111_T485-R1', 4], ['7373_T346-R1', 2], ['6924_T375-R1', 5], ['6939_T332-R1', 5], ['9543_T510-R1', 5]]



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/clades/Int12715_16/Int12715_16.transcript.fa
found all genes supplied; 34 genes


Exons added per gene: 
[['9100_T355-R1', 7], ['9537_T367-R1', 4], ['7416_T352-R1', 4], ['9580_T310-R1', 4], ['9792_T410-R1', 4], ['7328_T646-R1', 3], ['6924_T317-R1', 6], ['7213_T307-R1', 4], ['7167_T433-R1', 3], ['7322_T302-R1', 5], ['9597_T370-R1', 4], ['9543_T359-R1', 4], ['7417_T181-R1', 4], ['6899_T359-R1', 3], ['9583_T240-R1', 4], ['7111_T294-R1', 4], ['1925_T406-R1', 5], ['9550_T406-R1', 4], ['5784_T228-R1', 4], ['9332_T309-R1', 4], [

found all genes supplied; 57 genes


Exons added per gene: 
[['7322_T293-R1', 6], ['7288_T155-R1', 5], ['6974_T096-R1', 5], ['7373_T297-R1', 4], ['9871_T346-R1', 6], ['5784_T058-R1', 5], ['9597_T105-R1', 5], ['7273_T300-R1', 5], ['7413_T093-R1', 5], ['9887_T383-R1', 5], ['9762_T130-R1', 5], ['9545_T388-R1', 5], ['7213_T067-R1', 5], ['6909_T086-R1', 6], ['9549_T256-R1', 5], ['6899_T080-R1', 5], ['9764_T499-R1', 5], ['7416_T054-R1', 5], ['9100_T348-R1', 5], ['9792_T138-R1', 5], ['9580_T442-R1', 5], ['9879_T093-R1', 5], ['9537_T081-R1', 5], ['9543_T357-R1', 5], ['9134_T322-R1', 4], ['1925_T157-R1', 5], ['9669_T340-R1', 5], ['9600_T335-R1', 5], ['ATHALIANA_AT2G17060_1', 1], ['9557_T410-R1', 5], ['9583_T314-R1', 5], ['6906_T114-R1', 5], ['9533_T364-R1', 5], ['9554_T265-R1', 5], ['10015_T054-R1', 5], ['9784_T074-R1', 5], ['9550_T104-R1', 5], ['7186_T285-R1', 5], ['9518_T358-R1', 5], ['5993_T040-R1', 5], ['9536_T174-R1', 5], ['9869_T315-R1', 5], ['6939_T346-R1', 5], ['9654_T083-R1', 5], ['741

found all genes supplied; 59 genes


Exons added per gene: 
[['7373_T213-R1', 1], ['7273_T159-R1', 3], ['9543_T189-R1', 1], ['7288_T220-R1', 1], ['7111_T467-R1', 1], ['5993_T202-R1', 1], ['9557_T276-R1', 1], ['7322_T231-R1', 1], ['9869_T106-R1', 1], ['9762_T216-R1', 1], ['7167_T111-R1', 1], ['1925_T451-R1', 1], ['9332_T320-R1', 1], ['9583_T243-R1', 1], ['6939_T730-R1', 1], ['9784_T225-R1', 1], ['9533_T154-R1', 1], ['9764_T193-R1', 1], ['9100_T216-R1', 1], ['9537_T127-R1', 1], ['7413_T205-R1', 1], ['9545_T207-R1', 1], ['108_T368-R1', 1], ['9580_T542-R1', 1], ['ATHALIANA_AT5G47250_1', 1], ['9550_T269-R1', 1], ['9600_T175-R1', 2], ['9549_T094-R1', 1], ['6899_T281-R1', 1], ['5784_T221-R1', 1], ['9597_T341-R1', 1], ['9944_T143-R1', 1], ['6906_T273-R1', 1], ['7186_T236-R1', 1], ['9134_T222-R1', 1], ['9610_T189-R1', 1], ['7396_T216-R1', 1], ['9887_T251-R1', 1], ['6924_T283-R1', 1], ['7308_T102-R1', 1], ['6981_T252-R1', 1], ['7417_T144-R1', 1], ['9518_T182-R1', 1], ['9792_T402-R1', 1], ['9721_

It would be helpful to copy the best alignment file into the created clade folder.

In [8]:
for file in cleanedfiles: 
    clade=file.split('/')[-1].strip('.afa')
    input_file = "/global/scratch/users/chandlersutherland/e14/NLRCladeFinder/Atha_NLRome/" + file
    output_dir = "/global/scratch/users/chandlersutherland/e14/popgen/clades/"+clade
    os.popen('cp '+input_file+' '+output_dir)

In [9]:
cleanedfiles

['RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10172_376_423_R_43.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10172_376_424_R_11.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10172_376_435_R_60.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10172_376_495_R_128.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10172_376_624_R_44.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10547_34_50_L_17.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10547_34_50_R_17.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10637_304_324_L_31.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10637_304_324_R_203.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10637_304_326_R_37.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10637_304_362_R_19.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int10637_304_546_R_11.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int11061_33_35_L_29.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int11061_33_35_R_4.afa',
 'RAxML_tree_pbNB-ARC/Clade_Refinement_1/

Similarly for the tree output 

In [11]:
for file in cleanedfiles: 
    subfolder=file.split('/')[-2]
    clade=file.split('/')[-1].strip('.afa')
    input_file = "/global/scratch/users/chandlersutherland/e14/NLRCladeFinder/Atha_NLRome/RAxML_tree_pbNB-ARC/" + subfolder+'/RAxML_bipartitionsBranchLabels.'+clade+'.Raxml.out'
    output_dir = "/global/scratch/users/chandlersutherland/e14/popgen/clades/"+clade
    os.popen('cp '+input_file+' '+output_dir)

Create a single fasta file with all the NLR proteins in the NLRome

In [18]:
all_genes=gene.loc[:,'Gene']

In [20]:
#does not have to be clade aware, just making a single fasta
# create output fasta file 
g = open('/global/scratch/users/chandlersutherland/e14/popgen/NLRome_proteins.fasta', "w")

for gene in all_genes:
    count=0
    protein=[]
    line_number=0
    
    #convert gene name 
    if gene.startswith('ATHALIANA'):
        #using just the primary transcript
        clean=gene.replace('ATHALIANA_', '').replace('_1', '')+'.1'
    else:
        clean=gene.replace('_1', '.1').replace('_2', '.2').replace('_3', '.3')
    
    #search file with clean gene name 
    with open("/global/scratch/users/chandlersutherland/e14/hvnlr_clades/hvNLR/Athaliana_NLR_Phylogeny/Proteomes/NLRome.aa.fa",'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
            line_number += 1
            if line.find(clean) != -1: 
                protein.append(lines[line_number].strip('\n'))
                count += 1

        found.append(gene)
        g.writelines('>'+gene+'\n')
        g.writelines(''.join(protein)+'\n')

g.close()

#check that output transcripts matches number of input 
s=set(found)
q=set(list(gene_list))
if s == q:
      print('found all genes supplied; '+str(len(s))+' genes\n\n')
else:
      print('error, some genes missing')
      temp3 = [x for x in q if x not in s]
      print(str(temp3)+' not found')
        
print('\n\n')

g.close()
print('Gene file written to '+'/global/scratch/users/chandlersutherland/e14/popgen/NLRome_proteins.fasta')

error, some genes missing
[] not found



Gene file written to /global/scratch/users/chandlersutherland/e14/popgen/NLRome_proteins.fasta
